In [24]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import cloudscraper
from datetime import date
import random 
import psycopg2 as ps

url = "https://www.urbanoutfitters.com/" 
scraper = cloudscraper.create_scraper()

# agent rotating 
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]
user_agent = user_agent_list[random.randint(0, len(user_agent_list)-1)]
headers = {'User-Agent': user_agent}

response = scraper.get(url , headers = headers)
if not response.status_code == 200:
    None
try:
    results_page = BeautifulSoup(response.content,'lxml') 
except:
    None


In [25]:
# scrap products from first category (women's)
catg = results_page.find_all('a',class_ = 'c-pwa-header-navigation__link js-nav-link c-pwa-link c-pwa-link--client')[0].get('href')
url_new = url +  catg
soup = BeautifulSoup(scraper.get(url + catg).content,'lxml')

'https://www.urbanoutfitters.com//womens-clothing'

In [ ]:
# all products from the first page
product_all = soup.find_all('a',class_ = 'o-pwa-product-tile__link c-pwa-link c-pwa-link--client')
grab = []
count = 0

In [54]:
# def a func that grab all needed info for each product
# because we cannot vist websites many times in very short time
# so I only grabed 20 items

def grab_prodcuts():
    for i in product_all:
        while count <= 20:
            product = i.get('href')
            product_url = url + product 
            product = BeautifulSoup(scraper.get(url + product).content,'lxml')


            name = product.find('head').find('title').get_text()[:-19]

            material_temp = product.find('div', class_ = 's-pwa-cms c-pwa-markdown').find('p').findNext('p').get_text().replace('Content + Care','').split('-')
            material = ''
            for i in material_temp:
                if '%' in i:
                    material = material+i

            color = product.find_all('img', {'class' : 'o-pwa-color-swatch o-pwa-color-swatch--large'})[0].get('alt')

            size = []
            sizelist = product.find_all('label', {'class' : 'c-pwa-radio-boxes__label c-pwa-custom-radio__label c-pwa-custom-radio__label--default'})
            for i in range(1,len(sizelist)):
                size.append(product.find_all('label', {'class' : 'c-pwa-radio-boxes__label c-pwa-custom-radio__label c-pwa-custom-radio__label--default'})[i].get_text().strip())

            price = product.find_all('span', class_="c-pwa-product-price__current")[0].get_text()

            image_link = []
            image_list = product.find_all('a',class_="c-pwa-image-viewer-thumbnails__link")
            for i in image_list:
                image_link.append(i.get('href'))

            try:
                brand = product.find_all('a', class_="c-pwa-product-partner-url__link c-pwa-link c-pwa-link--client")[0].get_text()[8:]
            except:
                brand = 'N/A'
            description = product.find('div', class_="s-pwa-cms c-pwa-markdown").find('p').get_text()

            date = date.today()

            category = product.find('meta', property="product:category").get('content')
            category = category.split('>')[1][1:]

            gender = product.find('meta', property="product:category").get('content')
            gender = gender.split('>')[0][:5]

            if category == 'Vintage':
                second_hand = True
            else:
                second_hand = False
            grab.append([name,material,color,size,price,product_url, image_link, brand, description, date, category, gender, second_hand])

        count += 0

In [55]:
#grab_prodcuts()
grab  #20 items in total

[['Out From Under Modern Love Corset',
  ' 82% Nylon, 18% spandex Lace: 87% Nylon, 13% spandex',
  'Washed Black',
  ['S', 'M', 'XL'],
  '$59.00',
  'https://www.urbanoutfitters.com//shop/out-from-under-modern-love-corset?category=womens-clothing&color=002&viewcode=b',
  ['https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_b?$xlarge$&fit=constrain&qlt=80&wid=640',
   'https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_d?$xlarge$&fit=constrain&qlt=80&wid=640',
   'https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_e?$xlarge$&fit=constrain&qlt=80&wid=640',
   'https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_f?$xlarge$&fit=constrain&qlt=80&wid=640'],
  'Out From Under',
  'Irresistible Out From Under corset made from a body-hugging stretch fabric with a fitted bodice and underwire cups. Decorated with lace detailing for a romantic finish and topped with adjustable spaghetti straps and hook-and-eye back closures.',
  datetime.d

In [53]:
# make the list into dataframe
df = pd.DataFrame(columns=['name','material','color','size','price','product_url', 'image_link', 'brand', 'description', 'date', 'category', 'gender', 'second_hand'])
for details in grab:
        try:
            df = pd.concat([df,pd.DataFrame(data={'name':[details[0]],
                            'material':[details[1]],
                            'color': [details[2]],
                           'size':[details[3]],
                           'price':[details[4]],
                           'product_url':[details[5]],
                           'image_link':[details[6]],
                           'brand':[details[7]],
                           'description':[details[8]],
                            'date':[details[9]],
                            'category':[details[10]], 
                            'gender':[details[11]], 
                            'second_hand':[details[12]], 
                                                 
                            })],ignore_index=True)
        except:
            print ('something weird is going on!')
            continue
df

,name,material,color,size,price,product_url,image_link,brand,description,date,category,gender,second_hand
0,Out From Under Modern Love Corset,"82% Nylon, 18% spandex Lace: 87% Nylon, 13% s...",Washed Black,"[S, M, XL]",$59.00,https://www.urbanoutfitters.com//shop/out-from...,[https://images.urbndata.com/is/image/UrbanOut...,Out From Under,Irresistible Out From Under corset made from a...,2022-06-02,Intimates + Lounge,Women,False
1,Pink Floyd 1977 Tour Crew Neck Sweatshirt,"50% Cotton, 50% polyester",Pink,[S/M],$79.00,https://www.urbanoutfitters.com//shop/pink-flo...,[https://images.urbndata.com/is/image/UrbanOut...,N/A,"Washed soft and faded with an overdyed finish,...",2022-06-02,Tops,Women,False
2,BDG Corduroy Cutoff Carpenter Short,100% Cotton,Light Brown,"[24, 25, 26, 27, 28, 29, 30, 31, 32]",$59.00,https://www.urbanoutfitters.com//shop/bdg-cord...,[https://images.urbndata.com/is/image/UrbanOut...,BDG,A pair of corduroy shorts from BDG. Relaxed fi...,2022-06-02,Bottoms,Women,False
3,BDG Carrie Denim Short — Light Wash,"99% Cotton, 1% lycra",Light Blue,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]",$59.00,https://www.urbanoutfitters.com//shop/bdg-carr...,[https://images.urbndata.com/is/image/UrbanOut...,BDG,A forever cool pair of denim shorts from BDG. ...,2022-06-02,Bottoms,Women,False
4,adidas 3-Stripes Pull-On Short,100% Polyester,Black,"[XS, S, M, L, XL]",$35.00,https://www.urbanoutfitters.com//shop/adidas-3...,[https://images.urbndata.com/is/image/UrbanOut...,adidas,Pull-on short from adidas decorated with signa...,2022-06-02,Bottoms,Women,False
5,Polo Ralph Lauren Preppy Bear Crew Sock,,White + Purple,[ONE SIZE],$9.00,https://www.urbanoutfitters.com//shop/polo-ral...,[https://images.urbndata.com/is/image/UrbanOut...,Polo Ralph Lauren,Polo Ralph Lauren's famous bear brings his pre...,2022-06-02,Accessories,Women,False
6,Daily Concepts Daily Feet Silicone Scrubber,,Leaves Of Life,[ONE SIZE],$20.00,https://www.urbanoutfitters.com//shop/daily-co...,[https://images.urbndata.com/is/image/UrbanOut...,Daily Concepts,Cleanse your feet and transport your mind with...,2022-06-02,Beauty,Women,False
7,Everist Compostable Konjac Sponge,100% Konjac plant fiber,Assorted,[ONE SIZE],$12.00,https://www.urbanoutfitters.com//shop/everist-...,[https://images.urbndata.com/is/image/UrbanOut...,Everist.,The Everist Konjac Sponge is made from 100% na...,2022-06-02,Beauty,Women,False
8,Luma & Leaf Afterglow Serum,,Assorted,[ONE SIZE],$20.00,https://www.urbanoutfitters.com//shop/luma-and...,[https://images.urbndata.com/is/image/UrbanOut...,Luma And Leaf,Let radiant skin reign with this antioxidant-r...,2022-06-02,Beauty,Women,False
9,Vagabond Shoemakers Evy Slip-On Sandal,,Pink,"[US 6/EU 36, US 7/EU 37, US 8/EU 38, US 9/EU 3...",$140.00,https://www.urbanoutfitters.com//shop/vagabond...,[https://images.urbndata.com/is/image/UrbanOut...,Vagabond Shoemakers,Evy is the edgy slip-in sandals with a minimal...,2022-06-02,Shoes,Women,False


In [191]:
# config of AWS RDS
host_name = 'database-urbanoutfitters.cr2idzextzgp.us-west-1.rds.amazonaws.com'
dbname = 'postgres'
port = '5432'
username = 'postgres'
password = '12345678'

In [192]:
# connect to database
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
 
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn


In [193]:
# create prducts table 
def create_table(curr, tablename):
    create_table_command = ('''CREATE TABLE IF NOT EXISTS products (
                            name VARCHAR(500) PRIMARY KEY,
                            material TEXT ,
                            color TEXT ,
                            size TEXT ,
                            price TEXT ,
                            product_url TEXT ,
                            image_link TEXT ,
                            brand TEXT , 
                            description TEXT, 
                            date DATE, 
                            category TEXT, 
                            gender TEXT, 
                            second_hand TEXT)''')
    curr.execute(create_table_command, [ps.extensions.AsIs(tablename)])


In [194]:
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [177]:
TABLE_NAME = "products"
create_table(curr,TABLE_NAME)

In [178]:
# def a func to insert single row's value
def insert_into_table(curr, name,material,color,size,price,product_url, image_link, \
                      brand, description, date, category, gender, second_hand):
    insert = ("""INSERT INTO products (name, material, color, size, price, product_url, image_link, \
                brand, description, date, category, gender, second_hand) \
                VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (name, material,color,size,price,product_url, image_link, brand, \
                     description, date, category, gender, second_hand)
    curr.execute(insert, row_to_insert)



In [179]:
# take each row of df as input of last func
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['name'], row['material'], 
        row['color'], row['size'], row['price'], 
        row['product_url'], row['image_link'],
        row['brand'], row['description'], 
        row['date'], row['category'], row['gender'], 
        row['second_hand'])


In [189]:
append_from_df_to_db(curr, df)
conn.commit()
curr.close()
conn.close()

In [195]:
# check db
curr = conn.cursor()
sql = 'SELECT * from products'
curr.execute(sql)
rows = curr.fetchall()
print(rows)

[('Out From Under Modern Love Corset', ' 82% Nylon, 18% spandex Lace: 87% Nylon, 13% spandex', 'Washed Black', '{S,M,XL}', '$59.00', 'https://www.urbanoutfitters.com//shop/out-from-under-modern-love-corset?category=womens-clothing&color=002&viewcode=b', '{https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_b?$xlarge$&fit=constrain&qlt=80&wid=640,https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_d?$xlarge$&fit=constrain&qlt=80&wid=640,https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_e?$xlarge$&fit=constrain&qlt=80&wid=640,https://images.urbndata.com/is/image/UrbanOutfitters/57537995_002_f?$xlarge$&fit=constrain&qlt=80&wid=640}', 'Out From Under', 'Irresistible Out From Under corset made from a body-hugging stretch fabric with a fitted bodice and underwire cups. Decorated with lace detailing for a romantic finish and topped with adjustable spaghetti straps and hook-and-eye back closures.', datetime.date(2022, 6, 2), 'Intimates + Lounge', 'Wo